In [49]:
import fitz
import re

In [50]:
doc = fitz.open('H332 X-PRESS MULHACEN 11.1.2022.pdf')

In [51]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 550)

In [70]:
total_starts = []   # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
total_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad

for page in doc:
    starts = page.search_for("Shipper :")

    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # append rect

    stops = page.search_for("Units :")

    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height)) # append rect

    words_in_page = page.get_text_words()

    for word in words_in_page:
        total_words.append([word[0], word[1] + total_height, word[2], word[3] + total_height, word[4]])

    pods = page.search_for('Port of discharge')
    
    for rect in pods:
        total_pods.append(rect + (0, total_height + 8.3467, 0, total_height + 21.1507)) #8.3467 och 21.1507 är antalet punkter bort från "Port of Discharge"


    total_height += page.rect.height # to be used for following page




In [59]:
print(len(total_pods))
print(total_pods)

22
[Rect(464.3999938964844, 49.79002380371094, 509.75396728515625, 57.91402053833008), Rect(464.3999938964844, 644.7900238037109, 509.75396728515625, 652.9140205383301), Rect(464.3999938964844, 1239.790023803711, 509.75396728515625, 1247.91402053833), Rect(464.3999938964844, 1834.790023803711, 509.75396728515625, 1842.91402053833), Rect(464.3999938964844, 2429.790023803711, 509.75396728515625, 2437.91402053833), Rect(464.3999938964844, 3024.790023803711, 509.75396728515625, 3032.91402053833), Rect(464.3999938964844, 3619.790023803711, 509.75396728515625, 3627.91402053833), Rect(464.3999938964844, 4214.790023803711, 509.75396728515625, 4222.91402053833), Rect(464.3999938964844, 4809.790023803711, 509.75396728515625, 4817.91402053833), Rect(464.3999938964844, 5404.790023803711, 509.75396728515625, 5412.91402053833), Rect(464.3999938964844, 5999.790023803711, 509.75396728515625, 6007.91402053833), Rect(464.3999938964844, 6594.790023803711, 509.75396728515625, 6602.91402053833), Rect(464.3

In [60]:
print(len(total_stops))
print(total_stops)

26
[Rect(223.55999755859375, 404.6467590332031, 249.42071533203125, 416.064697265625), Rect(223.55999755859375, 780.8767395019531, 249.42071533203125, 792.2946929931641), Rect(223.55999755859375, 981.1067199707031, 249.42071533203125, 992.524658203125), Rect(223.55999755859375, 1375.8767395019531, 249.42071533203125, 1387.294692993164), Rect(223.55999755859375, 1595.1267395019531, 249.42071533203125, 1606.544677734375), Rect(223.55999755859375, 1942.3767395019531, 249.42071533203125, 1953.794692993164), Rect(223.55999755859375, 2142.5667419433594, 249.42071533203125, 2153.9846801757812), Rect(223.55999755859375, 2489.786750793457, 249.42071533203125, 2501.204689025879), Rect(223.55999755859375, 2699.5267333984375, 249.42071533203125, 2710.9446716308594), Rect(223.55999755859375, 2918.7767944335938, 249.42071533203125, 2930.1947021484375), Rect(223.55999755859375, 3303.5567321777344, 249.42071533203125, 3314.9746704101562), Rect(223.55999755859375, 3755.876739501953, 249.42071533203125,

In [64]:
def create_large_rect_of_pod():
    lista_pod_start = []
    lista_pod_end = []
    rektanglar_pod = []
    pod_namn = []

    for pod in total_pods:
        lista_pod_start.append(round(pod[1]))
        lista_pod_end.append(round(pod[1]+8))
        
    for num in range(len(lista_pod_start)-1):
        if num == 0:
            rektanglar_pod.append([1, lista_pod_start[num] , 600, lista_pod_start[num + 1]])
            continue

        rektanglar_pod.append([1, lista_pod_end[num] , 600, lista_pod_start[num + 1]])

    for word in words_in_page:
        if fitz.Rect(word[:4]).intersects(total_pods):
            pod_namn.append(word[4])

    print(word)
    print(rektanglar_pod)
    print()
    print(lista_pod_start)
    print(lista_pod_end)
        
    #print(lista_pod_start)
    #print(lista_pod_end)
    

create_large_rect_of_pod()


ValueError: bad sequ length

In [71]:
def gather_data(start, stop):
    rektangel = (464, start[1], 555, stop[3]) # rektangel för området
    samma_rad = start[1]        # värdet uppdateras varje loop med värdet från föregående loop
    lista_gathered_data = []    # varje ny rad representerar en instans i listan
    ordlista = ""
    
    for word in total_words:
            
        if fitz.Rect(word[:4]).intersects(rektangel):
            
            if samma_rad != word[1]:
                lista_gathered_data.append(ordlista.strip())
                ordlista = ""
                ordlista = word[4]

            elif samma_rad == word[1]:
                ordlista += " " + word[4]

            samma_rad = word[1]

    #lista_gathered_data.append(ordlista.strip()) # lägger till den sista raden i loopen
       
    return lista_gathered_data

In [76]:
for i, j in enumerate(total_pods):
    print(gather_data(total_pods[i]))

TypeError: gather_data() missing 2 required positional arguments: 'start' and 'stop'